In [18]:
import cv2
import numpy as np

In [19]:
labels = open('obj.names.txt').read().strip().split('\n')#reading pretrained object labels

In [20]:
weights = 'yolov4-obj_last.weights'#pretrained weights
architecture = 'yolov4-obj.cfg.txt'#Neural network architecture

In [21]:
model = cv2.dnn.readNet(architecture,weights)#reading the neural network

In [22]:
image = cv2.imread('car.jpg')#reading static image
Height,Width = image.shape[:2]#for scaling our bounding box wrt to image

In [23]:
blob = cv2.dnn.blobFromImage(image, 1/255.0, (608, 608), swapRB=True, crop=False)
blob.shape

(1, 3, 608, 608)

In [24]:
model.setInput(blob)

In [25]:
def forward_prop(model):
    layers = model.getLayerNames()#get all layer names
    output_layer = [layers[i[0]-1] for i in model.getUnconnectedOutLayers()]#get names of output layers
    return model.forward(output_layer)#does forward prop and gives output for the layers passed to it

In [26]:
CONFIDENCE = 0.5
IOU_THRESHOLD = 0.5
boxes, confidences, class_ids = [], [], []

In [27]:
outputs = forward_prop(model)#outputs of the last layer of NN 

In [28]:
for output in outputs:#loop through output of NN
    for pred in output:#loop through all predictions
        class_scores = pred[5:]#get class probs
        class_id = np.argmax(class_scores)#find max class prob
        class_prob = class_scores[class_id]#get class_id of class with max prob
        if class_prob>CONFIDENCE:
            #scale dimensions of bounding box wrt to image
            center_x = int(pred[0] * Width)
            center_y = int(pred[1] * Height)
            w = int(pred[2] * Width)
            h = int(pred[3] * Height)
            #getting top-left co-ordinates
            x = center_x - (w / 2)
            y = center_y - (h / 2)
            class_ids.append(class_id)
            confidences.append(float(class_prob))
            boxes.append([x, y, w, h])

In [29]:
indices=cv2.dnn.NMSBoxes(boxes,confidences,CONFIDENCE,IOU_THRESHOLD)#performs nms and returns boxes to be kept

In [30]:
def bounding_box(image,x,y,w,h,class_id,prob):
    label = labels[class_id]#getting label
    text = label+' : '+str(prob)
    cv2.rectangle(image,(x+2,y),(x+w-2,y+h),color=(255,0,0),thickness=2)

In [31]:
for i in indices.flatten():
    x,y,w,h = boxes[i][0],boxes[i][1],boxes[i][2],boxes[i][3]
    prob=round(confidences[i], 3)
    bounding_box(image,int(x),int(y),int(w),int(h), class_ids[i] ,prob )

In [32]:
print(x,y,x+w,y+h)

606.5 450.0 835.5 500.0


In [33]:
cv2.imshow("object detection", image)
cv2.waitKey()
cv2.destroyAllWindows()

In [34]:
cv2.imwrite("plate_1.jpg",image[int(y)+2:int(y)+int(h)-2,int(x)+2:int(x)+int(w)-2])

True